In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI矢量搜索查找StackOverflow问题

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_create_stack_overflow_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fvector_search%2Fsdk_vector_search_create_stack_overflow_embeddings.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/vector_search/sdk_vector_search_create_stack_overflow_embeddings.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_create_stack_overflow_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

该示例演示了如何使用StackOverflow数据集和句子-T5模型对自定义文本嵌入进行编码。这些嵌入被上传到Vertex AI矢量搜索服务。这是一种高规模、低延迟的解决方案，用于查找大语料库中相似向量（或更具体地说是“嵌入”）。此外，这是一种完全托管的产品，进一步降低了运营开销。Vertex AI矢量搜索服务是基于Google研究开发的[近似最近邻居（ANN）技术](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html)构建的。

**先决条件**：此笔记本要求您已经设置了一个VPC网络。请参阅[创建Vertex AI矢量搜索索引笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vector_search/sdk_vector_search_for_indexing.ipynb)中的“准备VPC网络”部分。

了解更多关于[Vertex AI矢量搜索](https://cloud.google.com/vertex-ai/docs/vector-search/overview)。

### 目标

在这本笔记本中，您将学习如何对自定义文本进行编码，创建近似最近邻居（ANN）索引，并对索引进行查询。

本教程使用以下Google Cloud ML服务：

- Vertex AI 向量搜索

执行的步骤包括：

* 创建ANN索引。
* 使用VPC网络创建索引端点。
* 部署ANN索引。
* 执行在线查询。

### 数据集

本教程使用的数据集是[StackOverflow数据集](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow)。

> Stack Overflow是程序员学习、分享知识和促进职业发展的最大在线社区。这个BigQuery数据集每季度更新一次，包括Stack Overflow内容的归档，包括帖子、投票、标签和徽章。该数据集更新以反映互联网档案馆上的Stack Overflow内容，也可通过Stack Exchange数据浏览器访问。

开始吧

### 为Python安装Vertex AI SDK和其他所需的软件包

In [ ]:
# Install the google-cloud packages
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage \
                         'google-cloud-bigquery[pandas]' -q

# Install the latest version of tensorflow packages
! pip3 install --upgrade tensorflow \
                         tensorflow_text \
                         tensorflow-hub -q

# Install the redis and tqdm packages
! pip install --upgrade redis \
                        tqdm -q

重新启动运行时（仅适用于Colab）

为了使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。在继续下一步之前，请等待其完成。⚠️</b>
</div>

将您的笔记本环境进行认证（仅限Colab）

在Google Colab上认证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置谷歌云项目信息

了解有关如何[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的更多信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 创建云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 初始化 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有一个现有的 Google Cloud 项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 导入所需的库

In [ ]:
import json
import os
import tempfile
from typing import List

import numpy as np
import redis
import tensorflow as tf
import tensorflow_hub as hub
# Registers the ops.
import tensorflow_text as text  # noqa: F401
from google.cloud import bigquery
from tqdm.auto import tqdm

准备数据

在本教程中，使用托管在BigQuery上的问题和答案的 [Stack Overflow数据集](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow)。

> 此公共数据集托管在Google BigQuery中，包含在BigQuery的每月免费1TB处理层中。每个用户每个月可以获得1TB的免费BigQuery处理能力，可用于对此公共数据集运行查询。

从BigQuery数据源提取数据集。

In [ ]:
%%time
client = bigquery.Client(project=PROJECT_ID)

NUM_ROWS = 1000

QUERY = f"""
        SELECT distinct q.id, q.title, q.body, q.tags, a.body as answers, a.score 
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q 
        INNER JOIN (SELECT * FROM `bigquery-public-data.stackoverflow.posts_answers`  where Score>0 ORDER BY Score desc) AS a ON q.id = a.parent_id 
        where q.tags like '%python%'
        LIMIT {NUM_ROWS};
        """

query_job = client.query(QUERY)
rows = query_job.result()

In [ ]:
# Convert to a dataframe
df = rows.to_dataframe()

# Examine the data
df.head()

In [ ]:
# Extract the question ids and question text
ids = df.id.tolist()
questions = df.title.tolist()

# Verify the length
len(ids)

### 实例化文本编码模型

使用谷歌开发的[sentence-t5编码器](https://tfhub.dev/google/sentence-t5/st5-base/1)将文本转换为嵌入向量。

句子-T5系列模型将文本编码为高维向量，可用于文本分类、语义相似性、聚类和其他自然语言处理任务。

该模型构建在T5（即文本到文本传输Transformer）之上。它经过多种数据源训练，并从具有不同模型大小的预训练T5模型进行初始化。输入是可变长度的英文文本，输出是一个768维向量。句子-T5基础模型采用12层Transformer架构，与T5基础模型相同。

In [ ]:
hub_url = "https://tfhub.dev/google/sentence-t5/st5-base/1"

encoder = hub.KerasLayer(hub_url)

定义一个编码函数

定义一个函数，稍后会用到，它将接受句子并将它们转换为嵌入。

In [ ]:
def encode_text_to_embedding(
    text_encoder: hub.KerasLayer, sentences: List[str], batch_size: int = 100
) -> np.ndarray:
    embeddings_list = []

    # Process data in chunks to prevent out-of-memory errors
    for i in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[i : i + batch_size]
        embeddings_list.append(text_encoder(tf.constant(batch)))

    return np.squeeze(np.column_stack(embeddings_list))

测试编码函数

编码一部分数据，看看嵌入和距离度量是否合理。

根据[sentence-T5研究论文](https://arxiv.org/pdf/2108.08877.pdf)，嵌入的相似性是使用点积来计算的。

In [ ]:
# Encode 500 questions
questions = df.title.tolist()[:500]
question_embeddings = encode_text_to_embedding(
    text_encoder=encoder, sentences=questions
)

在创建索引时，保存维度尺寸以供以后使用。

In [ ]:
DIMENSIONS = len(question_embeddings[0])

print(DIMENSIONS)

In [ ]:
question_index = 0

# Print the query question
print(f"Query question = {questions[question_index]}")
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

将列车分割保存为JSONL格式。

数据必须按照JSONL格式进行格式化，这意味着每个嵌入字典都被写成一个JSON字符串，各自占据一行。

有关更多信息，请参阅[input data format and structure](https://cloud.google.com/vertex-ai/docs/vector-search/setup/format-structure#data-file-formats)中的文档。

In [ ]:
# Create temporary file to write embeddings to
embeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)

print(embeddings_file.name)

In [ ]:
# Set batch size
BATCH_SIZE = 100

# Create embeddings and write to a file
with open(embeddings_file.name, "a") as f:
    for i in tqdm(range(0, len(questions), BATCH_SIZE)):
        id_chunk = ids[i : i + BATCH_SIZE]

        question_chunk_embeddings = encode_text_to_embedding(
            text_encoder=encoder, sentences=questions[i : i + BATCH_SIZE]
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk, question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

将训练数据上传到云存储桶。

In [ ]:
UNIQUE_FOLDER_NAME = "embeddings_folder_unique"
remote_folder = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp {embeddings_file.name} {remote_folder}

创建索引##

创建ANN指数（用于生产使用）

In [ ]:
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "questions from stackoverflow"

创建ANN索引配置：

要了解有关配置索引的更多信息，请参阅[向量搜索输入数据格式和结构](https://cloud.google.com/vertex-ai/docs/vector-search/setup/setup)。

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=remote_folder,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=80,
    description=DESCRIPTION,
)

In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
print(INDEX_RESOURCE_NAME)

使用资源名称，您可以检索现有的MatchingEngineIndex资源。

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

## 设置VPC对等网络

要使用矢量搜索索引，请在您的项目和Vertex AI矢量搜索服务项目之间设置VPC对等网络。这可以消除网络流量中的额外跳跃，并允许使用高效的gRPC协议。

了解更多关于[VPC对等](https://cloud.google.com/vertex-ai/docs/general/vpc-peering)。

**重要提示：每个项目只能设置与servicenetworking.googleapis.com的一个VPC对等连接。**

### 创建VPC对等连接

为简便起见，设置VPC对等连接到`ucaip-haystack-vpc-network`网络。您可以为您的项目创建一个不同的网络。

如果您与任何其他网络建立VPC对等连接，请确保该网络已经存在，并且您的虚拟机正在该网络上运行。

In [ ]:
# This is for display only; you can name the range anything.
NETWORK = "ucaip-haystack-vpc-network"  # @param {type:"string"}
PEERING_RANGE_NAME = "vertex-ai-prediction-peering-range"

# NOTE: `prefix-length=16` means a CIDR block with mask /16 is
# reserved for use by Google services, such as Vertex AI.
! gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service" \
  --network=$NETWORK \
  --purpose=VPC_PEERING

### 创建VPC连接

接下来，为VPC对等连接创建连接。

**注意:** 如果收到 PERMISSION DENIED 错误，可能是您的默认服务帐户没有设置必要的“Compute Network Admin”角色。在 Cloud 控制台中，执行以下操作：

1. 转到 **IAM & Admin**。
2. 找到您的服务帐户。
3. 点击编辑图标。
4. 选择 **Add Another Role**。
5. 输入 **Compute Network Admin**。
6. 选择 **Save**。

In [ ]:
! gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

检查您的对等连接状态。

In [ ]:
! gcloud compute networks peerings list --network $NETWORK

构建完整的网络名称

在随后为VPC对等创建Vector Search索引端点资源时，您需要拥有完整的网络资源名称。

In [ ]:
# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

full_network_name = f"projects/{PROJECT_NUMBER}/global/networks/{NETWORK}"
print(full_network_name)

使用VPC网络创建IndexEndpoint。

In [ ]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    network=full_network_name,
)

部署索引

### 部署ANN索引

In [ ]:
# Set a unique id for your deployed index
DEPLOYED_INDEX_ID = "deployed_index_id_unique"

In [ ]:
# Deploy your ANN index to the index endpoint
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

## 创建在线查询

在构建索引之后，您可以查询已部署的索引以查找最近邻。

**注意：** 对于 **DOT_PRODUCT_DISTANCE** 距离类型，每个MatchNeighbor返回的“distance”属性实际上指的是相似度。

In [ ]:
test_embeddings = encode_text_to_embedding(
    text_encoder=encoder, sentences=["How do I install tensorflow with GPU support?"]
)

In [ ]:
# Test query
NUM_NEIGHBOURS = 20

response = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=[test_embeddings.tolist()],
    num_neighbors=NUM_NEIGHBOURS,
)

response

打印标题以验证邻居是否合理

In [ ]:
neighbor_ids = [neighbor.id for neighbor in response[0]]
neighbor_distances = [neighbor.distance for neighbor in response[0]]

for match_index, neighbor in enumerate(response[0]):
    titles = df[df.id.astype(str) == neighbor.id].title.tolist()

    if len(titles) > 0:
        print(
            f"{match_index}: title = '{titles[0]}', distance = {neighbor.distance:0.2f}"
        )

将标题存储和检索到Redis数据存储中
当您将此代码投入生产并转换为服务时，您需要将从Vertex AI矢量搜索返回的最近的ids转换为可用于下游服务的数据。

在这种情况下，您需要将ids转换为标题。

您可以使用Google Cloud的Memorystore部署一个托管的Redis实例来保存id-title键值对。

有关更多信息，请参阅[Memorystore](https://cloud.google.com/memorystore/docs/redis/create-manage-instances?hl=zh-CN)。

In [ ]:
# Set a display name for your Redis instance
REDIS_INSTANCE_NAME = "stackoverflow-questions-unique"

# Create a Redis instance
! gcloud redis instances create '{REDIS_INSTANCE_NAME}' --size=5 --region={LOCATION} --network={VPC_NETWORK_FULL} --connect-mode=private-service-access

In [ ]:
# Get host and port info
if not os.getenv("IS_TESTING"):
    REDIS_HOST = ! gcloud redis instances list --filter="INSTANCE_NAME:'{REDIS_INSTANCE_NAME}'" --region {LOCATION}  --format='value(HOST)'
    REDIS_PORT = ! gcloud redis instances list --filter="INSTANCE_NAME:'{REDIS_INSTANCE_NAME}'" --region {LOCATION} --format='value(PORT)'

    if isinstance(REDIS_HOST, list):
        REDIS_HOST = REDIS_HOST[0]

    if isinstance(REDIS_PORT, list):
        REDIS_PORT = REDIS_PORT[0]

    print(f"REDIS_HOST = {REDIS_HOST}")
    print(f"REDIS_PORT = {REDIS_PORT}")

In [ ]:
# Connect to the instance
if not os.getenv("IS_TESTING"):
    redis_client = redis.StrictRedis(host=REDIS_HOST, port=REDIS_PORT)

In [ ]:
# Convert the id -> title relationship into a dict and write to redis
if not os.getenv("IS_TESTING"):
    redis_client.mset({str(id): str(title) for id, title in zip(df.id, df.title)})

In [ ]:
# Verify that redis can retrieve the correct information
if not os.getenv("IS_TESTING"):
    [
        f"Actual = {title}, Retrieved = {redis_client.get(str(id))}"
        for id, title in list(zip(df.id, df.title))[:10]
    ]

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。
您还可以通过运行以下代码手动删除创建的资源。

In [ ]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

# Delete indexes
tree_ah_index.delete()

# Delete cloud storage bucket
delete_bucket = False
if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}

# Delete redis instance
! gcloud redis instances delete '{REDIS_INSTANCE_NAME}' --region {LOCATION} --quiet